In [1]:
from yugiquery import *
header('Rush Duel')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Rush Duel - Execution started 17/02/2023 21:11 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

# Data aquisition

## Fetch online data

In [2]:
# Timestamp
timestamp = pd.Timestamp.now()

In [3]:
# Fetch rush cards
rush_df = fetch_rush(debug = True)

KeyError: 'printouts'

## Save data

In [ ]:
rush_df.to_csv(f'../data/all_rush_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
print('Data saved')

# Check changes

## Load previous data

In [ ]:
# Get list of files
files_list = sorted(glob.glob('../data/all_rush_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if len(files_list)>1:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[1], dtype=object, keep_default_na=False, na_values=[''])
    # Correct tuples
    previous_df['Effect type'] = previous_df['Effect type'].dropna().apply(literal_eval)
    previous_df['Archseries'] = previous_df['Archseries'].dropna().apply(literal_eval)
    previous_df['Artwork'] = previous_df['Artwork'].dropna().apply(literal_eval)
    # previous_df['Misc'] = previous_df['Misc'].dropna().apply(literal_eval)
    # previous_df['Errata'] = previous_df['Errata'].dropna().apply(literal_eval)
    # Force dtypes to match current df
    previous_df = previous_df.astype(rush_df[previous_df.columns].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[1]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

## Generate changelog

In [ ]:
if previous_df is None:
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, rush_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/rush_changelog_{timestamp.isoformat(timespec="minutes")}_{previous_ts.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

# Data visualization

In [ ]:
rush_df

## Card type

In [ ]:
rush_df['Card type'].nunique()

In [ ]:
card_type_colors = [colors_dict[i] for i in rush_df['Card type'].value_counts().index]
rush_df['Card type'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=0, color = card_type_colors, title = 'Card type')
plt.show()

## Attribute

In [ ]:
rush_df['Attribute'].nunique()

In [ ]:
rush_df['Attribute'].value_counts()

## Primary type

In [ ]:
rush_df['Primary type'].nunique()

In [ ]:
rush_df['Primary type'].value_counts()

## Monster type

In [ ]:
rush_df['Monster type'].nunique()

In [ ]:
rush_df['Monster type'].value_counts()

## Level/Rank type

In [ ]:
rush_df['Level/Rank'].nunique()

In [ ]:
rush_df['Level/Rank'].value_counts()

## ATK

In [ ]:
rush_df['ATK'].nunique()

In [ ]:
rush_df['ATK'].value_counts()

## DEF

In [ ]:
rush_df['DEF'].nunique()

In [ ]:
rush_df['DEF'].value_counts()

## Effect type

In [ ]:
rush_df['Effect type'].explode().nunique()

In [ ]:
rush_df['Effect type'].explode().value_counts()

## Archseries

In [ ]:
rush_df['Archseries'].explode().nunique()

In [ ]:
rush_df['Archseries'].explode().value_counts()

## Legend

In [ ]:
rush_df['Legend'].sum()

In [ ]:
rush_df[rush_df['Legend']]

## Maximum mode

In [ ]:
rush_df['Maximum mode'].sum()

In [ ]:
rush_df[rush_df['Maximum mode']]

In [ ]:
### MAXIMUM ATK

In [ ]:
rush_df['MAXIMUM ATK'].count()

## Artworks

In [ ]:
rush_df['Artwork'].nunique()

In [ ]:
rush_df['Artwork'].value_counts()

## Property

In [ ]:
rush_df['Property'].nunique()

In [ ]:
rush_df['Property'].value_counts()

In [ ]:
rush_df.iloc[pd.to_numeric(rush_df['DEF'], errors='coerce').argmax()]

# Epilogue

In [ ]:
footer(timestamp)

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Rush.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Rr]ush*"

In [ ]:
! git commit -m {"'Rush duel update-" + timestamp.isoformat() + "'"}